# Installation

In [ ]:
!uv add langgraph langchain langsmith python-dotenv langchain_ollama langchain_community langchain_tavily

In [ ]:
## load env variables
from dotenv import load_dotenv

load_dotenv()

### Tavily Tool

In [ ]:
## Create tavily tool
from langchain_tavily import TavilySearch

search_tool=TavilySearch(max_results=2)

### Custom Multiply Tool

In [ ]:
## custom function
def multiply(a:int,b:int)->int:
    """Multiply a and b

    Args:
        a (int): first int if it is in string convert to int
        b (int): second int if it is in string convert to int

    Returns:
        int: result of the multiplication
    """
    return a * b

### Create LLM using Ollama

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2", temperature=0)

In [ ]:
## add tools in llm 
tools=[search_tool, multiply]
llm_with_tools=llm.bind_tools(tools)

### Creating the Chatbot

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

## State 
class State(TypedDict):
    messages:Annotated[list, add_messages]

## Node functionality
def chatbot(state:State):
    return {"messages": [llm.invoke(state["messages"])]}

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

## Node definition
def tool_calling_llm(state:State):
    return {"messages":[llm_with_tools.invoke(state["messages"])]}

## Graph
builder=StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools",ToolNode(tools))

## Add Edges
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges("tool_calling_llm",tools_condition)
builder.add_edge("tools",END)

## Compile the graph
graph = builder.compile()

In [ ]:
graph

### Using this Graph

In [ ]:
response=graph.invoke({"messages":"What is the recent AI news for today?"})

for m in response["messages"]:
    m.pretty_print()

In [ ]:
response=graph.invoke({"messages":"What is 25 multiply by 5?"})

for m in response["messages"]:
    m.pretty_print()

In [ ]:
response=graph.invoke({"messages":"What is 10 multiply by 5 and then multiply by 2?"})

for m in response["messages"]:
    m.pretty_print()

### ReAct Agent 

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

## Node definition
def tool_calling_llm(state:State):
    return {"messages":[llm_with_tools.invoke(state["messages"])]}

## Graph
builder=StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools",ToolNode(tools))

## Add Edges
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges("tool_calling_llm",tools_condition)
builder.add_edge("tools", "tool_calling_llm")

## Compile the graph
graph = builder.compile()

In [ ]:
graph

In [ ]:
response=graph.invoke({"messages":"Give me the recent AI news for today and then multiply 10 by 5?"})

for m in response["messages"]:
    m.pretty_print()

In [ ]:
response=graph.invoke({"messages":"What is 10 multiply by 5 and then multiply by 2?"})

for m in response["messages"]:
    m.pretty_print()

### Adding Memory In Agentic Graph

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
tools=[multiply]
llm_with_tools=llm.bind_tools(tools)

## Node definition
def tool_calling_llm(state:State):
    return {"messages":[llm_with_tools.invoke(state["messages"])]}

## Graph
builder=StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools",ToolNode(tools))

## Add Edges
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges("tool_calling_llm",tools_condition)
builder.add_edge("tools", "tool_calling_llm")

## Compile the graph
graph = builder.compile(checkpointer=memory)
graph

In [ ]:
config={"configurable":{"thread_id":"1"}}

response=graph.invoke({"messages":"Hi, My name is Rahul. How are you?"},config=config)

for m in response["messages"]:
    m.pretty_print()

In [ ]:
response=graph.invoke({"messages":"Do you remember my name?"},config=config)

response["messages"][-1].content